# 隐马尔可夫模型

隐马尔可夫模型由初始状态分布、状态转移概率分布和观测概率分布确定  
其形式定义如下  
Q是所有可能的状态集合，V是所有可能的观测的集合    
N是可能的状态数，M是可能的观测数
$$Q = \{q_1,q_2,...,q_N\}$$  
$$V=\{v_1,v_2,...,v_M\}$$

I是长度为T的状态序列，O是对应的观测序列  
$$I=(i_1,i_2,...,i_T)$$  
$$O=(o_1,o_2,...,o_T)$$  

A是状态转移矩阵  
$$ A=[a_{ij}]_{NxN} $$  
其中，$a_{ij}=P(i_{t+i}=q_j|i_t=q_i)$， $i=1,2,...N;j=1,2,...N$

B是观测概率矩阵  
$$B=[b_j(k)]_{NxN}$$  
其中，$b_j(k)=P(o_t=v_k|i_t=q_j)$, $k=1,2,...M;j=1,2,...N$
是在时刻t处于状态$q_j$的条件下生成观测$v_k$的概率  

$\pi$是初始状态概率向量  
$$\pi=(\pi_i)$$
其中，$\pi_i=P(i_1=q_i)$,$i=1,2,...N$  
是t=1处于状态$q_i$的概率  

隐马尔可夫模型有初始状态概率向量$\pi$、状态转移概率矩阵A和观测概率矩阵B决定。  
$\pi$和A决定状态序列，B决定观测序列  
因此，隐马尔可夫模型$\lambda$可以用三元符号表示  
$$\lambda=(A,B,\pi)$$

In [1]:
# 公式书上都有
import numpy as np
class HiddenMarkov:
    def forward(self, Q, V, A, B, O, PI):
        N = len(Q) # 可能的状态数  
        M = len(V) # 可能的观测数
        alphas = np.zeros((N, M)) 
        T = M # 有多少观测序列就有多少时刻  
        for t in range(T):
            index_O = V.index(O[t]) # 找出对应观测序列上当前的观测的在观测集中索引
            for i in range(N): # 对不同的状态
                if t==0:  # 初始状态，pi
                    alphas[i][t] = PI[t][i] * B[i][index_O]
                    print('alpha1(%d)=p%db%db(o1)=%f'%(i,i,i,alphas[i][t]))
                else:
                    alphas[i][t] = np.dot([alpha[t-1] for alpha in alphas], [a[i] for a in A]) * B[i][index_O]
                    print('alpha%d(%d)=[sigma alpha%d(i)ai%d]b%d(o%d)=%f' % (t, i, t - 1, i, i, t, alphas[i][t]))
                    
        P = np.sum([alpha[M-1] for alpha in alphas])
        
    def backward(self, Q, V, A, B, O, Pi):
        N = len(Q) # 可能的状态数
        M = len(O) # 观测序列的大小  
        betas = np.ones((N, M))
        for i in range(N):
            print('betas%d(%d)=1'%(M,i))
        for t in range(M-2, -1, -1):
            index_O = V.index(O[t+1])
            for i in range(N):
                betas[i][t] = np.dot(np.multiply(A[i], [b[index_O] for b in B]), [beta[t+1] for beta in betas])
                realT = t+1
                realI = i+1
                print('beta%d(%d)=[sigma a%djbj(o%d)]beta%d(j)'%(realT, realI, realI, realT+1, realT+1), end='')
                for j in range(N):
                    print('%.2f*%.2f%.2f+'%(A[i][j], B[j][index_O], betas[j][t+1]),end='')
                print('0)%.3f'%betas[i][t])
        index_O = V.index(O[0])
        P = np.dot(np.multiply(PI, [b[index_O] for b in B]),[beta[0] for beta in betas])
        print('P(O|lambda)=', end='')
        for i in range(N):
            print('%.1f*%.1f*%.5f+' % (Pi[0][i], B[i][index_O], betas[i][0]),end='')
        print('0=%f' % P)
        
                  
    def viterbi(self, Q, V, A, B, O, PI):
        N = len(Q)  #可能存在的状态数量
        M = len(O)  # 观测序列的大小
        deltas = np.zeros((N, M))
        psis = np.zeros((N, M))
        I = np.zeros((1, M))
        for t in range(M):
            realT = t + 1
            indexOfO = V.index(O[t])  # 找出序列对应的索引
            for i in range(N):
                realI = i + 1
                if t == 0:
                    deltas[i][t] = PI[0][i] * B[i][indexOfO]
                    psis[i][t] = 0
                    print('delta1(%d)=pi%d * b%d(o1)=%.2f * %.2f=%.2f' %
                          (realI, realI, realI, PI[0][i], B[i][indexOfO],
                           deltas[i][t]))
                    print('psis1(%d)=0' % (realI))
                else:
                    deltas[i][t] = np.max(
                        np.multiply([delta[t - 1] for delta in deltas],
                                    [a[i] for a in A])) * B[i][indexOfO]
                    print(
                        'delta%d(%d)=max[delta%d(j)aj%d]b%d(o%d)=%.2f*%.2f=%.5f'
                        % (realT, realI, realT - 1, realI, realI, realT,
                           np.max(
                               np.multiply([delta[t - 1] for delta in deltas],
                                           [a[i] for a in A])), B[i][indexOfO],
                           deltas[i][t]))
                    psis[i][t] = np.argmax(
                        np.multiply(
                            [delta[t - 1] for delta in deltas],
                            [a[i]
                             for a in A])) + 1  #由于其返回的是索引，因此应+1才能和正常的下标值相符合。
                    print('psis%d(%d)=argmax[delta%d(j)aj%d]=%d' % (realT, realI, realT - 1, realI, psis[i][t]))
        print(deltas)
        print(psis)
        I[0][M - 1] = np.argmax([delta[M - 1] for delta in deltas
                                 ]) + 1  #由于其返回的是索引，因此应+1才能和正常的下标值相符合。
        print('i%d=argmax[deltaT(i)]=%d' % (M, I[0][M - 1]))
        for t in range(M - 2, -1, -1):
            I[0][t] = psis[int(I[0][t + 1]) - 1][t + 1]
            print('i%d=psis%d(i%d)=%d' % (t + 1, t + 2, t + 2, I[0][t]))
        print("状态序列I：", I)
            
        
        
        
        

In [2]:
#习题10.1
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
# O = ['红', '白', '红', '红', '白', '红', '白', '白']
O = ['红', '白', '红', '白']    #习题10.1的例子
PI = [[0.2, 0.4, 0.4]]

In [3]:
HMM = HiddenMarkov()
# HMM.forward(Q, V, A, B, O, PI)
# HMM.backward(Q, V, A, B, O, PI)
HMM.viterbi(Q, V, A, B, O, PI)

delta1(1)=pi1 * b1(o1)=0.20 * 0.50=0.10
psis1(1)=0
delta1(2)=pi2 * b2(o1)=0.40 * 0.40=0.16
psis1(2)=0
delta1(3)=pi3 * b3(o1)=0.40 * 0.70=0.28
psis1(3)=0
delta2(1)=max[delta1(j)aj1]b1(o2)=0.06*0.50=0.02800
psis2(1)=argmax[delta1(j)aj1]=3
delta2(2)=max[delta1(j)aj2]b2(o2)=0.08*0.60=0.05040
psis2(2)=argmax[delta1(j)aj2]=3
delta2(3)=max[delta1(j)aj3]b3(o2)=0.14*0.30=0.04200
psis2(3)=argmax[delta1(j)aj3]=3
delta3(1)=max[delta2(j)aj1]b1(o3)=0.02*0.50=0.00756
psis3(1)=argmax[delta2(j)aj1]=2
delta3(2)=max[delta2(j)aj2]b2(o3)=0.03*0.40=0.01008
psis3(2)=argmax[delta2(j)aj2]=2
delta3(3)=max[delta2(j)aj3]b3(o3)=0.02*0.70=0.01470
psis3(3)=argmax[delta2(j)aj3]=3
delta4(1)=max[delta3(j)aj1]b1(o4)=0.00*0.50=0.00189
psis4(1)=argmax[delta3(j)aj1]=1
delta4(2)=max[delta3(j)aj2]b2(o4)=0.01*0.60=0.00302
psis4(2)=argmax[delta3(j)aj2]=2
delta4(3)=max[delta3(j)aj3]b3(o4)=0.01*0.30=0.00220
psis4(3)=argmax[delta3(j)aj3]=3
[[0.1      0.028    0.00756  0.00189 ]
 [0.16     0.0504   0.01008  0.003024]
 [0.28     0.

In [4]:
# 例10.2
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
O = ['红', '白', '红', '红', '白', '红', '白', '白']
PI = [[0.2, 0.3, 0.5]]

In [5]:
HMM.forward(Q, V, A, B, O, PI)
HMM.backward(Q, V, A, B, O, PI)

alpha1(0)=p0b0b(o1)=0.100000
alpha1(1)=p1b1b(o1)=0.120000
alpha1(2)=p2b2b(o1)=0.350000
alpha1(0)=[sigma alpha0(i)ai0]b0(o1)=0.078000
alpha1(1)=[sigma alpha0(i)ai1]b1(o1)=0.111000
alpha1(2)=[sigma alpha0(i)ai2]b2(o1)=0.068700
betas8(0)=1
betas8(1)=1
betas8(2)=1
beta7(1)=[sigma a1jbj(o8)]beta8(j)0.50*0.501.00+0.20*0.601.00+0.30*0.301.00+0)0.460
beta7(2)=[sigma a2jbj(o8)]beta8(j)0.30*0.501.00+0.50*0.601.00+0.20*0.301.00+0)0.510
beta7(3)=[sigma a3jbj(o8)]beta8(j)0.20*0.501.00+0.30*0.601.00+0.50*0.301.00+0)0.430
beta6(1)=[sigma a1jbj(o7)]beta7(j)0.50*0.500.46+0.20*0.600.51+0.30*0.300.43+0)0.215
beta6(2)=[sigma a2jbj(o7)]beta7(j)0.30*0.500.46+0.50*0.600.51+0.20*0.300.43+0)0.248
beta6(3)=[sigma a3jbj(o7)]beta7(j)0.20*0.500.46+0.30*0.600.51+0.50*0.300.43+0)0.202
beta5(1)=[sigma a1jbj(o6)]beta6(j)0.50*0.500.21+0.20*0.400.25+0.30*0.700.20+0)0.116
beta5(2)=[sigma a2jbj(o6)]beta6(j)0.30*0.500.21+0.50*0.400.25+0.20*0.700.20+0)0.110
beta5(3)=[sigma a3jbj(o6)]beta6(j)0.20*0.500.21+0.30*0.400.25+0.50*